In [1]:
import numpy as np
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend

import json
import re

Using TensorFlow backend.


# Load Posts and Comments from data

In [2]:
def preproc(s):
    s = re.sub(r'\[.*\|.*\]', ' ', s)
    s = re.sub(r'\s+', ' ', s)
    s = re.sub(r'[«»]', '"', s)
    return s.lower()

In [3]:
with open('data/kalikfan.json', 'r') as f:
    kalikfan = json.load(f)
    
text = ''.join([preproc(x['text']) for x in kalikfan])
text

'деньги в кармане жена на диване сын на борьбе житуха по кайфу)-милый смотри какую я скульптуру слепила из кальянчика) -дорогая, а что это😅 -как что? мотогонщик - вон шлем у него надет) -ааааа, мотогонщик) а я уж напридумывал всякого🤣 нормальная девушка) воображуля одним словом😌 выставила свою поделку на , надеюсь ей поставят высший балл)вот вы думаете, почему ваши родители с таким теплом вспоминают пионерские деньки🤔 из-за того, что ездили в какой-нибудь лагерь типа "орленок" или "артек"? или из-за пилоток? неееееет😀 ошибаетесь) все потому, что только пионерам доставалась самая улётная забивка, ядерная - советская😎 только по кумарному госту🤙🏼о май гарабл, даниила) этот кальянчик просто супер дыымабл😈 русский красавелла) просто крэйзи, ахах😅 девушки, вот вам жених, все подруги обзавидуются) хватит вам искать себе избранника в фитнес зале, настоящие самцы тут - в кальянных😎вот часто так бывает сидите в калике дуете по тихому и охота прикола какого-то) анекдотов поблизости нет, камеди не

# Shisha Learning 🤙🤙🤙
From https://stackabuse.com/text-generation-with-python-and-tensorflow-keras/

In [4]:
chars = sorted(list(set(text)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

input_len = len(text)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 75950
Total vocab: 172


In [5]:
seq_length = 20
x_data = []
y_data = []

# loop through inputs, start at the beginning and go until we hit
# the final character we can create a sequence out of
for i in range(0, input_len - seq_length, 1):
    # Define input and output sequences
    # Input is the current character plus desired sequence length
    in_seq = text[i:i + seq_length]

    # Out sequence is the initial character plus total sequence length
    out_seq = text[i + seq_length]

    # We now convert list of characters to integers based on
    # previously and add the values to our lists
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 75930


In [6]:
X = np.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [7]:
y = np_utils.to_categorical(y_data)

In [8]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [10]:
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [11]:
model.fit(X, y, epochs=20, batch_size=256, callbacks=desired_callbacks)

Train on 75930 samples
Epoch 1/20


75776/75930 [============================>.] - ETA: 0s - loss: 3.4087
Epoch 00001: loss improved from inf to 3.40842, saving model to model_weights_saved.hdf5
75930/75930 [==============================] - 13s 176us/sample - loss: 3.4084
Epoch 2/20
75264/75930 [============================>.] - ETA: 0s - loss: 3.3237
Epoch 00002: loss improved from 3.40842 to 3.32395, saving model to model_weights_saved.hdf5
75930/75930 [==============================] - 9s 120us/sample - loss: 3.3239
Epoch 3/20
75776/75930 [============================>.] - ETA: 0s - loss: 3.3153
Epoch 00003: loss improved from 3.32395 to 3.31547, saving model to model_weights_saved.hdf5
75930/75930 [==============================] - 8s 111us/sample - loss: 3.3155
Epoch 4/20
75264/75930 [============================>.] - ETA: 0s - loss: 3.3092
Epoch 00004: loss improved from 3.31547 to 3.30973, saving model to model_weights_saved.hdf5
75930/75930 [==============================] - 7s 95us/sample - loss: 3.3097
Epoch 5

In [12]:
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [13]:
num_to_char = dict((i, c) for i, c in enumerate(chars))

In [14]:
def take_seed():
    start = np.random.randint(0, len(kalikfan) - 1)
    s = preproc(kalikfan[start]['text'])[:seq_length]
    return [char_to_num[x] for x in s]

In [15]:
def pred(pattern):
    result = []
    for i in range(100):
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(vocab_len)
        prediction = model.predict(x, verbose=0)
        index = np.argmax(prediction)
        result += num_to_char[index]
        seq_in = [num_to_char[value] for value in pattern]

        pattern.append(index)
        pattern = pattern[1:len(pattern)]
        
    return ''.join(result)

In [16]:
def to_input(s):
    return [char_to_num[x] for x in s]

In [17]:
def test():
    s = take_seed()
    p = pred(s)
    print(''.join([num_to_char[x] for x in s]) + "|" + p)

In [19]:
test()

-да я тебе говорю - -|-да па по па по пооери па пооери па пооери па пооери па пооери па пооери па пооери па пооери па поое
